# Working with APIs in Python

Making API requests in Python can be really simple. There's a low-level module called urllib that can also make the kinds of web requests that we want, but it's not as friendly as the `requests` module, which we'll be using.

In [1]:
import requests

## Authentication

You'll have to authenticate each request to the Harvard Art Museum API with an API key. Other APIs may require different kinds of authentication (sometimes very complicated auth! Look for libraries at that point), but HAM has some pretty simple authentication, which makes things easy for us. You can sign up for a key [here](https://www.harvardartmuseums.org/collections/api).

In [2]:
APIKEY = "b0cde630-ce66-11e8-951c-b3d75228cc98" # Enter your API key here

## Basic request

We're going to start off with a basic request to the API. This API, like many others, has a variety of endpoints, each with their own url, slightly modified from a base url. We'll worry about the general case in a bit, for now let's look at a basic API request.

In this example, we'll re-create the first example in the [Object endpoint documentation](https://github.com/harvardartmuseums/api-docs/blob/master/object.md), which will give each of you the records for 10 objects that have never been viewed online in the museum's collections.

In [4]:
url = "https://api.harvardartmuseums.org/object"
parameters = {
    "q":"totalpageviews:0",
    "size":10,
    "apikey":APIKEY
}
R = requests.get(url,params=parameters)
R.json()

{'info': {'next': 'https://api.harvardartmuseums.org/object?q=totalpageviews%3A0&size=10&apikey=b0cde630-ce66-11e8-951c-b3d75228cc98&page=2',
  'page': 1,
  'pages': 5630,
  'totalrecords': 56293,
  'totalrecordsperquery': 10},
 'records': [{'accessionmethod': 'Gift',
   'accessionyear': 1970,
   'accesslevel': 1,
   'century': None,
   'classification': 'Fragments',
   'classificationid': 94,
   'colorcount': 0,
   'commentary': None,
   'contact': 'am_asianmediterranean@harvard.edu',
   'contextualtextcount': 0,
   'copyright': None,
   'creditline': 'Harvard Art Museums/Arthur M. Sackler Museum, Gift of Dr. George C. Scanlon',
   'culture': 'Egyptian',
   'datebegin': 0,
   'dated': None,
   'dateend': 0,
   'dateoffirstpageview': None,
   'dateoflastpageview': None,
   'department': 'Department of Islamic & Later Indian Art',
   'description': None,
   'dimensions': None,
   'division': 'Asian and Mediterranean Art',
   'edition': None,
   'exhibitioncount': 0,
   'groupcount': 0,


### Refresher on Dictionaries

Python dictionaries are sets of key / value pairs, where a value can be accessed by its key. You're essentially naming a value in a container, so you can easily call it up later.

Dictionaries have very fast lookups, so you can get a value from its key very quickly, no matter how large the dictionary is. However, they are also unordered, so if you iterate through all of the key / value pairs in the dictionary, there's no guarantee that they'll be in the same order.

We're just going to be looking up data in dictionaries, so here's a quick refresher on the syntax:

In [5]:
parameters['q']

'totalpageviews:0'

In [6]:
parameters['apikey'] # This also works when we've set the value to another variable

'b0cde630-ce66-11e8-951c-b3d75228cc98'

In [7]:
parameters['q'] = "totalpageviews:1" # You can also set the value of a key like you would a variable

## Making a Request

The request syntax is so simple, you might have missed it. Let's query again for objects with only one pageview, and take a closer look.

In [8]:
R = requests.get(url,params=parameters)

### Formatted parameters

That request has created a request object, which contains not only the data that we get from the Harvard Art Museums, but information on the request we sent, like the URL that it used. Notice that requests has turned our query parameter dictionary into a GET request at the end of our URL.

If you've been working with API requests or web scraping before, you might be used to seeing URLs get constructed like this:

```python
url = "https://api.harvardartmuseums.org/object?q=" + query + "&apikey=" + apikey
```

If you have, I'm sure you'll appreciate how much simpler this is, especially when dealing with more query parameters.

In [10]:
R.url

'https://api.harvardartmuseums.org/object?q=totalpageviews%3A1&size=10&apikey=b0cde630-ce66-11e8-951c-b3d75228cc98'

### Taking a look at the results

Request objects have a built-in method, `.json()`, which converts a JSON file received as a response to a request from a string of text that happens to be in this data format into Python native data structures, like lists, dictionaries, numbers and strings. We can use this method to see a dictionary representation of what we've gotten from the API request.

In [11]:
R.json()

{'info': {'next': 'https://api.harvardartmuseums.org/object?q=totalpageviews%3A1&size=10&apikey=b0cde630-ce66-11e8-951c-b3d75228cc98&page=2',
  'page': 1,
  'pages': 2396,
  'totalrecords': 23960,
  'totalrecordsperquery': 10},
 'records': [{'accessionmethod': 'Transfer',
   'accessionyear': 2011,
   'accesslevel': 1,
   'century': '20th century',
   'classification': 'Photographs',
   'classificationid': 17,
   'colorcount': 0,
   'commentary': None,
   'contact': 'am_moderncontemporary@harvard.edu',
   'contextualtextcount': 0,
   'copyright': None,
   'creditline': 'Harvard Art Museums/Fogg Museum, Transfer from the Carpenter Center for the Visual Arts, American Professional Photographers Collection',
   'culture': 'American',
   'datebegin': 1945,
   'dated': 'c. 1950',
   'dateend': 1955,
   'dateoffirstpageview': '2012-08-24',
   'dateoflastpageview': '2012-08-24',
   'department': 'Department of Photographs',
   'description': None,
   'dimensions': 'image: 10.16 x 12.7 cm (4 x 

## Changing our request

Let's say we're not interested in the most obscure parts of the collection (pot sherds, apparently), but rather in the most popular parts of the collection. There are a few ways we might go about doing this. One way might be to sort our search results by `totalpageviews`, and see what the top 10 are.

To do that, we can go back to the API documentation and look for hints about what we might be able to do